Insert explanatory text here. Allow 5 minutes for completion.

In [1]:
load("preamble.sage")

Construct the set of $\mathbb{F}_2$-rational points of $\mathbf{P}(1:1:1:2)$.

In [2]:
F = GF(2)
P.<x0,x1,x2,y> = PolynomialRing(F, 4)

In [3]:
S = [vector(F,t) for t in itertools.product(F, repeat=4) if not all(i==0 for i in t)]
for v in S:
    v.set_immutable()
len(S)

15

Construct pairs of cubic and quartic hypersurfaces in $\mathbb{P}(1:1:1:2)$ whose $\mathbb{F}_2$-rational points match a given 6-tuple or 7-tuple not including the singular point $[0:0:0:1]$.

In [4]:
gens1 = [a+b for a in [0, x0*y] for b in [x1*x2*(x1+x2), x1*(x1^2+x1*x2+x2^2), x1^3+x1*x2^2+x2^3]]
mons4 = [prod(P.gens()[i]^w[i] for i in range(4)) for w in WeightedIntegerVectors(4, [1,1,1,2])]

In [5]:
coords4 = {x: vector(mu(*x) for mu in mons4) for x in S}

In [6]:
def redundancy(gen1, P=P, F=F, mons4=mons4):
    tmp = [gen1*x for x in P.gens()[:3]]
    return [vector(F, (gen.coefficient(mu) for mu in mons4)) for gen in tmp]

In [7]:
l = []
V0 = VectorSpace(F, len(mons4))
for v1 in gens1:
    pts = [x for x in S if v1(*x) == 0]
    perp = Matrix([coords4[x] for x in pts])
    for ct in [6, 7]:
        for pts1 in itertools.combinations([x for x in pts if x != S[0]], ct):
            target = vector(F, (0 if x in pts1 else 1 for x in pts))
            for w2 in solve_right_iterator(perp, target, redundancy, v1):
                v2 = sum(w2[i]*mons4[i] for i in range(len(mons4)))
                l.append((ct,v1,v2))
len(l)

202496

Enforce the desired point counts over $\mathbb{F}_{2^i}$ for $i=2,3$.

In [8]:
def count_by_ideal(gens, n):
    J = P.ideal(list(gens) + [y^(2^n) + y for y in P.gens()])
    return (J.vector_space_dimension() - 1) // (2^n-1)

In [9]:
l1a = [vecs[1:] for vecs in l if vecs[0] == 6]
l1b = [vecs[1:] for vecs in l if vecs[0] == 7]
len(l1a), len(l1b)

(61952, 140544)

In [10]:
l2a = [vecs for vecs in l1a if count_by_ideal(vecs, 2) == 18]
len(l2a)

202

In [11]:
l2b = [vecs for vecs in l1b if count_by_ideal(vecs, 2) == 15]
len(l2b)

25350

In [12]:
l3a = [vecs for vecs in l2a if count_by_ideal(vecs, 3) == 12]
l3b = [vecs for vecs in l2b if count_by_ideal(vecs, 3) == 7]
len(l3a), len(l3b)

(48, 102)

Use Magma to compute the full zeta functions of the resulting curves and compare these to our targets, finding no results.

In [13]:
I1 = P.ideal([x0,x1,x2,y])
CR = magma.CoxRing(P, [I1], [[1,1,1,2]], [])
proj = CR.ToricVariety()
proj

Toric variety of dimension 3 over Finite field of size 2
Variables: x0, x1, x2, y
The irrelevant ideal is:
(y, x2, x1, x0)
The grading is:
1, 1, 1, 2

In [14]:
for gens in l3a:
    X = proj.Scheme(gens)
    if X.Dimension() > 1 or str(X.IsIrreducible()) == "false":
        continue
    C = X.Curve()
    ct = tuple(Integer(C.NumberOfPlacesOfDegreeOneECF(i)) for i in range(1, 8))
    assert ct not in targets7

In [15]:
print("This stage complete!")

This stage complete!
